# 2. Search Engine

In [3]:
import pandas as pd
import nltk
import pickle
from functions import change_columns,search_engine_1,vocabulary,tfidf
from search_engine_2 import search_engine_2

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kkfam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### DataFrame of all msc's

In [43]:
#import the tsv.file and store it in a pandas.dataframe
df= pd.read_csv("merged_tsv.tsv", sep = '\t')


>#### Make sure we extracted all the MSc's info

In [5]:
df.shape

(6000, 15)

> #### Change the df in order to match exactly with the github example 

In [44]:
df = change_columns(df)

In [6]:
df.head(10)

,courseName,universityName,facultyName,isItFullTime,courseDescription,startDate,fees,modality,duration,city,country,administration,url
0,Chemistry and Molecular Sciences - MSc,University of Helsinki,International Masters Degree Programmes,True,A Master’s degree in chemistry qualifies you f...,September,Tuition fee per year (non-EU/EEA students): 15...,MSc,2 years,Helsinki,Finland,On Campus,https://www.findamasters.com/masters-degrees/c...
1,Digital Business Management MSc,University of Portsmouth,Faculty of Business and Law,True,Digital technology is having a dramatic impact...,"September, January",NaN,MSc,"1 year full-time, 2 years part-time",Portsmouth,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
2,Master of Science in Computer Science,Albert Ludwig University of Freiburg,Department of Computer Science,True,The accredited Master of Science program in Co...,"October, April",For EU nationals: 155 Euro per semester For no...,MSc,2 years,Freiburg,Germany,On Campus,https://www.findamasters.com/masters-degrees/c...
3,Preventative Cardiovascular Medicine - MSc,Learna | Diploma MSc,Diploma MSc Medical Programmes,False,Preventative cardiovascular medicine is a rapi...,"March, September",£9000 (2 years),MSc,2 years,Cardiff,United Kingdom,Online,https://www.findamasters.com/masters-degrees/c...
4,Applied Marine and Fisheries Ecology MSc,Aberdeen University,School of Biological Sciences,True,Applied Marine and Fisheries Ecology gives you...,"September, January",Find out about fees: Please see the university...,MSc,"12 months Full Time, 24 months part time",Aberdeen,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
5,"Leadership, Communication and Humanitarian Cha...",University of Huddersfield,Huddersfield Business School,True,This course is specifically aimed at Internati...,"September, January",International fees: Please see the university ...,MSc,18 months full time with placement,Huddersfield,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
6,Advancing Practice Sensory Integration (MSc),Sheffield Hallam University,Postgraduate Courses,True,Develop your knowledge and understanding of th...,"January, April",Our tuition fee for UK students starting part-...,MSc,2 / 4 years distance learning,Sheffield,United Kingdom,Online,https://www.findamasters.com/masters-degrees/c...
7,Early Childhood Education - MS,New York Institute of Technology,Education,True,Teach young learners as they embark on a journ...,See Course,Please see the university website for further ...,MSc,See website for details.,New York,USA,Online,https://www.findamasters.com/masters-degrees/c...
8,International Tourism and Events Management - MSc,Glasgow Caledonian University,Glasgow School for Business and Society,True,Tourism and events are among the fastest growi...,"September, January",Please see the university website for further ...,MSc,16 months full time; 2 years (with Professiona...,Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
9,Advanced Manufacturing Systems MSc online,Brunel University Online,Brunel Online Masters Programmes,False,About the course OurAdvanced Manufacturing Sys...,"September, January","£10,605",MSc,2 years,London,United Kingdom,Online,https://www.findamasters.com/masters-degrees/c...


>#### We save the modified dataframe as csv file

In [7]:
directory_path = r'C:\Users\kkfam\Desktop\DATA_SCIENCE\ADM\HOMEWORKS\HOMEWORK_3\\'
csv_filename = 'df.csv'
full_path = directory_path + csv_filename

# Save the DataFrame to the specified CSV file
df.to_csv(full_path, index=False)

## 2.0 Preprocessing

### 2.0.0) Preprocessing the text

### Removing punctuation

>#### We use the nltk tokenize package to rid off all punctuation

In [7]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [45]:
# Remove punctuation for each column
for column in df.columns:
        if column != 'url' and column != 'fees':  # otherwise we lose the exact url
            df[column] = df[column].apply(lambda row: ' '.join(tokenizer.tokenize(str(row))) if isinstance(row, str) else row)
df.head(10)

,courseName,universityName,facultyName,isItFullTime,courseDescription,startDate,fees,modality,duration,city,country,administration,url
0,Chemistry and Molecular Sciences MSc,University of Helsinki,International Masters Degree Programmes,True,A Master s degree in chemistry qualifies you f...,September,Tuition fee per year (non-EU/EEA students): 15...,MSc,2 years,Helsinki,Finland,On Campus,https://www.findamasters.com/masters-degrees/c...
1,Digital Business Management MSc,University of Portsmouth,Faculty of Business and Law,True,Digital technology is having a dramatic impact...,September January,NaN,MSc,1 year full time 2 years part time,Portsmouth,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
2,Master of Science in Computer Science,Albert Ludwig University of Freiburg,Department of Computer Science,True,The accredited Master of Science program in Co...,October April,For EU nationals: 155 Euro per semester For no...,MSc,2 years,Freiburg,Germany,On Campus,https://www.findamasters.com/masters-degrees/c...
3,Preventative Cardiovascular Medicine MSc,Learna Diploma MSc,Diploma MSc Medical Programmes,False,Preventative cardiovascular medicine is a rapi...,March September,£9000 (2 years),MSc,2 years,Cardiff,United Kingdom,Online,https://www.findamasters.com/masters-degrees/c...
4,Applied Marine and Fisheries Ecology MSc,Aberdeen University,School of Biological Sciences,True,Applied Marine and Fisheries Ecology gives you...,September January,Find out about fees: Please see the university...,MSc,12 months Full Time 24 months part time,Aberdeen,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
5,Leadership Communication and Humanitarian Chal...,University of Huddersfield,Huddersfield Business School,True,This course is specifically aimed at Internati...,September January,International fees: Please see the university ...,MSc,18 months full time with placement,Huddersfield,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
6,Advancing Practice Sensory Integration MSc,Sheffield Hallam University,Postgraduate Courses,True,Develop your knowledge and understanding of th...,January April,Our tuition fee for UK students starting part-...,MSc,2 4 years distance learning,Sheffield,United Kingdom,Online,https://www.findamasters.com/masters-degrees/c...
7,Early Childhood Education MS,New York Institute of Technology,Education,True,Teach young learners as they embark on a journ...,See Course,Please see the university website for further ...,MSc,See website for details,New York,USA,Online,https://www.findamasters.com/masters-degrees/c...
8,International Tourism and Events Management MSc,Glasgow Caledonian University,Glasgow School for Business and Society,True,Tourism and events are among the fastest growi...,September January,Please see the university website for further ...,MSc,16 months full time 2 years with Professional ...,Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
9,Advanced Manufacturing Systems MSc online,Brunel University Online,Brunel Online Masters Programmes,False,About the course OurAdvanced Manufacturing Sys...,September January,"£10,605",MSc,2 years,London,United Kingdom,Online,https://www.findamasters.com/masters-degrees/c...


### Removing stopwords and Stemming

In [9]:
from nltk.stem import *
from nltk.corpus import stopwords

In [10]:
nltk.download('stopwords')
lst_stopwords = stopwords.words('english')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kkfam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
stemmer = PorterStemmer()    # we chose arbitrarly the Porter Stemmer among several others possible stemmers algorithms

In [46]:
for column in df.columns:
    if column != 'url' and column != 'fees':
        df[column]=df[column].apply(lambda row:' '.join([stemmer.stem(word) for word in str(row).split(' ') if not word in lst_stopwords]))


In [47]:
df['list_of_words'] = df.courseDescription.apply(lambda row: [word for word in row.split(' ')])
list_of_words = df['list_of_words'].copy()

>#### We save the column of the courseDescription preprocessed and made it into a list

In [14]:
directory_path = r'C:\Users\kkfam\Desktop\DATA_SCIENCE\ADM\HOMEWORKS\HOMEWORK_3\\'
csv_filename = 'df_list_of_words.csv'
full_path = directory_path + csv_filename

# Save the DataFrame to the specified CSV file
list_of_words.to_csv(full_path, index=False)

## 2.1. Conjunctive query

### 2.1.1) Create your index!

>#### First we create our vocabulary with all the words that appear in the courseDescription column. We achieve this task with a dictionary that maps each term to a different number

In [15]:
# Split the course descriptions into terms and create a vocabulary
terms = df['courseDescription'].str.split().explode().drop_duplicates()
vocabulary = dict(zip(terms, pd.factorize(terms)[0]))
vocabulary

{'a': 0,
 'master': 1,
 'degre': 2,
 'chemistri': 3,
 'qualifi': 4,
 'expert': 5,
 'posit': 6,
 'wide': 7,
 'rang': 8,
 'field': 9,
 'industri': 10,
 'research': 11,
 'educ': 12,
 'the': 13,
 'chemic': 14,
 'major': 15,
 'employ': 16,
 'one': 17,
 'largest': 18,
 'export': 19,
 'finland': 20,
 'your': 21,
 'work': 22,
 'could': 23,
 'also': 24,
 'involv': 25,
 'applic': 26,
 'environment': 27,
 'biolog': 28,
 'scienc': 29,
 'manufactur': 30,
 'pharmaceut': 31,
 'product': 32,
 'develop': 33,
 'technolog': 34,
 'materi': 35,
 'new': 36,
 'energi': 37,
 'solut': 38,
 'in': 39,
 'privat': 40,
 'sector': 41,
 'duti': 42,
 'might': 43,
 'includ': 44,
 'qualiti': 45,
 'manag': 46,
 'train': 47,
 'commerc': 48,
 'custom': 49,
 'forens': 50,
 'chemist': 51,
 'control': 52,
 'analys': 53,
 'sampl': 54,
 'part': 55,
 'often': 56,
 'requir': 57,
 'interdisciplinari': 58,
 'intern': 59,
 'cooper': 60,
 'as': 61,
 'invent': 62,
 'serv': 63,
 'connoisseur': 64,
 'natur': 65,
 'phenomena': 66,
 'digi

>#### and we save it as pkl file

In [18]:
directory_path = r'C:\Users\kkfam\Desktop\DATA_SCIENCE\ADM\HOMEWORKS\HOMEWORK_3\\'
csv_filename = 'vocabulary.pkl'
file_path = directory_path + csv_filename

with open(file_path, 'wb') as file:
    pickle.dump(vocabulary, file)

>#### Now we implement the Inverted Index, adding a new column 'reverse' in the terms dataframe that for each row is a list with all the document_id  containing the respective term (row <-> term)

### Inverted Index

In [31]:
# Function to update inverted index for each row
def update_inverted_index(row,inverted_index):
        
    document_id = row.name  # Get the document ID (DataFrame index)
    terms = row['list_of_words']
    for term in terms:
        term_id = vocabulary.get(term)
        if term_id is not None:
            if document_id not in inverted_index[term_id]:
                inverted_index[term_id].append(document_id)


# Initialize the inverted index dictionary
inverted_index = {term_id: [] for term_id in vocabulary.values()}
            
# Apply the function to update the inverted index
df.apply(update_inverted_index, args=(inverted_index,), axis=1);

In [32]:
print(inverted_index)

{0: [0, 40, 47, 63, 67, 72, 120, 124, 180, 200, 215, 231, 292, 299, 323, 385, 395, 418, 422, 451, 454, 613, 623, 647, 667, 704, 783, 825, 882, 890, 909, 924, 927, 1017, 1021, 1063, 1082, 1093, 1149, 1182, 1185, 1197, 1248, 1251, 1290, 1379, 1406, 1438, 1486, 1520, 1649, 1700, 1705, 1746, 1797, 1864, 1906, 1925, 1953, 1996, 2020, 2057, 2104, 2167, 2247, 2264, 2265, 2287, 2343, 2370, 2373, 2377, 2555, 2667, 2686, 2696, 2712, 2726, 2829, 2837, 2850, 2902, 2903, 2918, 2938, 3030, 3086, 3113, 3114, 3115, 3142, 3193, 3215, 3221, 3357, 3366, 3377, 3395, 3401, 3439, 3453, 3458, 3519, 3561, 3579, 3613, 3617, 3666, 3705, 3707, 3717, 3738, 3786, 3812, 3873, 3992, 4002, 4010, 4026, 4035, 4055, 4070, 4153, 4186, 4240, 4265, 4312, 4314, 4329, 4357, 4364, 4391, 4398, 4457, 4505, 4650, 4689, 4708, 4844, 4935, 4978, 4979, 5015, 5042, 5045, 5054, 5112, 5124, 5129, 5154, 5176, 5181, 5184, 5205, 5231, 5239, 5266, 5304, 5314, 5415, 5416, 5420, 5424, 5433, 5440, 5531, 5592, 5596, 5650, 5686, 5734, 5766, 596

>#### and we save it in a local directory using the pickle module

In [20]:
directory_path = r'C:\Users\kkfam\Desktop\DATA_SCIENCE\ADM\HOMEWORKS\HOMEWORK_3\\'
csv_filename = 'inverted_index.pkl'
file_path = directory_path + csv_filename

with open(file_path, 'wb') as file:
    pickle.dump(inverted_index, file)

### 2.1.2) Execute the query

In [40]:
unprocessed_df = pd.read_csv(r"C:\Users\kkfam\Desktop\DATA_SCIENCE\ADM\HOMEWORKS\HOMEWORK_3\df.csv")  #we load the dataframe

>#### Since we do not want to compute the inverted index every time we use the Search Engine, we have stored it in a separate file and now we load it in memory.

In [21]:
with open(r"C:\Users\kkfam\Desktop\DATA_SCIENCE\ADM\HOMEWORKS\HOMEWORK_3\inverted_index.pkl", 'rb') as file:
    inverted_index = pickle.load(file)
    

In [28]:
print(inverted_index)

{0: [0, 40, 47, 63, 67, 72, 120, 124, 180, 200, 215, 231, 231, 292, 299, 323, 385, 395, 418, 422, 451, 454, 613, 623, 647, 667, 704, 783, 825, 882, 890, 890, 909, 924, 927, 1017, 1021, 1063, 1082, 1093, 1149, 1182, 1185, 1197, 1248, 1251, 1290, 1379, 1406, 1438, 1486, 1520, 1649, 1700, 1705, 1746, 1746, 1797, 1864, 1906, 1925, 1953, 1996, 2020, 2020, 2057, 2104, 2167, 2247, 2264, 2265, 2287, 2343, 2370, 2373, 2377, 2555, 2667, 2686, 2696, 2712, 2726, 2829, 2837, 2850, 2850, 2902, 2903, 2918, 2938, 3030, 3086, 3113, 3114, 3115, 3142, 3193, 3215, 3221, 3357, 3366, 3377, 3395, 3401, 3439, 3453, 3458, 3519, 3561, 3579, 3613, 3617, 3666, 3705, 3707, 3717, 3738, 3786, 3812, 3873, 3992, 3992, 4002, 4010, 4026, 4035, 4055, 4070, 4153, 4186, 4240, 4265, 4312, 4314, 4329, 4357, 4364, 4391, 4398, 4457, 4505, 4650, 4650, 4689, 4708, 4844, 4935, 4978, 4979, 5015, 5042, 5045, 5054, 5112, 5124, 5129, 5154, 5176, 5181, 5184, 5205, 5231, 5239, 5266, 5304, 5314, 5415, 5416, 5420, 5424, 5433, 5440, 5440,

>#### we also stored the vocabulary, so now we load it 

In [33]:
with open(r"C:\Users\kkfam\Desktop\DATA_SCIENCE\ADM\HOMEWORKS\HOMEWORK_3\vocabulary.pkl", 'rb') as file:
    vocabulary = pickle.load(file)

In [41]:
doc_found, _ = search_engine_1(df = unprocessed_df, inverted_index= inverted_index, vocabulary = vocabulary)

Make a query: Medicine and Biology


In [42]:
doc_found

,courseName,universityName,courseDescription,url
2944,Drug Discovery and Development MSc,Aberdeen University,"If you are interested in how drugs metabolise,...",https://www.findamasters.com/masters-degrees/c...
5254,Advanced Biochemistry MSc,University of Strathclyde,You'll be studying an MSc in Advanced Biochemi...,https://www.findamasters.com/masters-degrees/c...
1286,Systems and Control MSc,University of Groningen,"Flying to the moon, monitoring and stabilizing...",https://www.findamasters.com/masters-degrees/c...
4616,Applied Statistics - MSc,Xi’an Jiaotong-Liverpool University,Applied Statistics (Biostatistics) is an impor...,https://www.findamasters.com/masters-degrees/c...
2822,Cellular and Molecular Medicine - MSc by Research,University of Bristol,TheSchool of Cellular and Molecular Medicineis...,https://www.findamasters.com/masters-degrees/c...
2698,Master of Science in Biomedical Technology - M...,Tampere University,Insights into the molecular mechanisms and arc...,https://www.findamasters.com/masters-degrees/c...
3979,Master of Science in Biomedical Technology - B...,Tampere University,Informatics solutions for the future of biomed...,https://www.findamasters.com/masters-degrees/c...
18,Advanced Biomedical Engineering - MSc,University of Bradford,Biomedical engineering is a fast evolving inte...,https://www.findamasters.com/masters-degrees/c...
5653,Human Movement Sciences MSc,University of Groningen,How does the nervous system control voluntary ...,https://www.findamasters.com/masters-degrees/c...
1557,Applied Genomics - MSc,Imperial College London,This Master’s course will teach you how to gen...,https://www.findamasters.com/masters-degrees/c...


## 2.2) Conjunctive query & Ranking score

### 2.2.1) Inverted index

>#### Now we compute the new inverted index adding the tfidf score for each term and document

In [48]:
new_tfidf_dict = tfidf(df,inverted_index,vocabulary)

>#### then we save it

In [ ]:
import pickle

directory_path = r'C:\Users\kkfam\Desktop\DATA_SCIENCE\ADM\HOMEWORKS\HOMEWORK_3\\'
csv_filename = 'new_inverted_index.pkl'
file_path = directory_path + csv_filename

with open(file_path, 'wb') as file:
    pickle.dump(new_tfidf_dict, file)

### 2.2.2) Execute the query

>#### For the second search engine, given a query, we want to get the top-k documents related to the query.

In [49]:
top_k = 5 #we choose a number

>#### For the same reason of point 2.1.2 we do not want neither to compute the tfidf inverted index every time we use the Search Engine, so we have stored it in a separate file and now we load it in memory.

In [50]:
with open(r"C:\Users\kkfam\Desktop\DATA_SCIENCE\ADM\HOMEWORKS\HOMEWORK_3\new_inverted_index.pkl", 'rb') as file:
    tfidf_inverted_index = pickle.load(file) 

In [54]:
topk_df = search_engine_2(df, vocabulary, inverted_index, tfidf_inverted_index, top_k)

Make a query: Medicine and Biology


In [55]:
topk_df

,courseName,universityName,courseDescription,url,Similarity
3822,master scienc biomed technolog cell technolog,tamper univers,in depth knowledg practic skill cell technolog...,https://www.findamasters.com/masters-degrees/c...,0.279238
2698,master scienc biomed technolog molecular biolog,tamper univers,insight molecular mechan architectur cell biom...,https://www.findamasters.com/masters-degrees/c...,0.273046
2381,msc biomed engin,univers sheffield,our msc design provid necessari technic knowle...,https://www.findamasters.com/masters-degrees/c...,0.258132
1609,biomed engin healthcar msc eng,univers liverpool,discov principl technolog led biomed engin bec...,https://www.findamasters.com/masters-degrees/c...,0.251296
4513,biomed engin manag healthcar msc,univers liverpool,discov principl technolog led biomed engin bec...,https://www.findamasters.com/masters-degrees/c...,0.246698
